In [41]:
import glob
liste_txt = glob.glob("tout_range/*.txt")
print(f"Nombre de txt : {len(liste_txt)}")

Nombre de txt : 41551


In [42]:
import re
def ouvrir_fichier(path):
    with open(path, encoding="utf-8") as f:
        chaine = f.read()
    return chaine

def decouper_mots(chaine):
    mots = [re.sub("\r|\n|\|","", mot) for mot in chaine.split()]
    return mots

def get_effectif(liste):
    dic = {}
    for mot in liste:
        dic.setdefault(mot, 0)
        dic[mot]+=1
    return dic

def merge_dic(dic1, dic2):
    communs   = set(dic1.keys()).intersection(set(dic2.keys()))
    dans_dic2 = set(dic2.keys()).difference(set(dic1.keys()))
    for cle in communs:
        dic1[cle]+=dic2[cle]
    for cle in dans_dic2:
        dic1[cle] = dic2[cle]
    return dic1


In [43]:
dic_effectifs_global = {}
for path in liste_txt[:5]:
    chaine = ouvrir_fichier(path)
    mots = decouper_mots(chaine)
    effectifs_fichier = get_effectif(mots)
    dic_effectifs_global = merge_dic(dic_effectifs_global, effectifs_fichier)
    print(f"Vocabulaire global : {len(dic_effectifs_global)}, Fichier courant : {len(effectifs_fichier)}")

Vocabulaire global : 87, Fichier courant : 87
Vocabulaire global : 87, Fichier courant : 0
Vocabulaire global : 122, Fichier courant : 44
Vocabulaire global : 144, Fichier courant : 23
Vocabulaire global : 151, Fichier courant : 10


In [44]:
def generer_lexique(dic_effectifs):
    NB_tokens = sum(dic_effectifs.values())
    dic_out ={mot: {"Effectif":effectif,"Fréquence":effectif/NB_tokens} for mot, effectif in dic_effectifs.items()}
    return dic_out

In [45]:
lexique_frequence = generer_lexique(dic_effectifs_global)

In [46]:
import json
import re
def stocker_lexique(dic, name_base="lexique"):
    with open(f"{name_base}.json", "w", encoding="utf-8") as w:
         w.write(json.dumps(dic, ensure_ascii=False, indent=2))
    voc_trie = sorted(list(dic.keys()))
    with open(f"{name_base}.txt", "w", encoding="utf-8") as w:
        for mot in voc_trie:
            w.write("|".join([mot, str(dic[mot]["Effectif"]), str(dic[mot]["Fréquence"])]) + "\n")

In [47]:
stocker_lexique(lexique_frequence)

In [48]:
with open("LGERM.json", encoding="utf-8") as f:
    LGERM = json.load(f)
mots_LGERM = set(LGERM)

In [49]:
dic_effectifs_global = {}
for path in liste_txt[:5000]:
    chaine = ouvrir_fichier(path)
    mots = decouper_mots(chaine)
    effectifs_fichier = get_effectif(mots)
    dic_effectifs_global = merge_dic(dic_effectifs_global, effectifs_fichier)
    
lexique_frequence = generer_lexique(dic_effectifs_global)
stocker_lexique(lexique_frequence, "lexique_complet")
print(f"Taille Voc : {len(lexique_frequence)}")


Taille Voc : 75203


In [50]:
communs   = set(lexique_frequence.keys()).intersection(mots_LGERM)
manquants = set(lexique_frequence.keys()).difference(mots_LGERM)

In [51]:
print(f"Taille LGERM : {len(LGERM)}")
print(f"Taille Voc : {len(lexique_frequence)}")
taux_lex = len(communs)/len(lexique_frequence)
print(taux_lex)

Taille LGERM : 3049125
Taille Voc : 75203
0.10980944909112668


In [52]:
NB_tokens = sum(dic_effectifs_global.values())
print(NB_tokens)

314408


In [53]:
# taux lexicalité pondéré
eff_commun = [dic_effectifs_global[x] for x in communs]
total = sum(eff_commun)
taux_lex_pond = total/NB_tokens
print(taux_lex_pond)

0.4896503905753034


In [54]:
l_communs = [[dic_effectifs_global[x], x] for x in communs]
print(sorted(l_communs, reverse=True)[:20])

[[13444, 'de'], [6499, '&'], [6400, 'la'], [6071, 'que'], [5478, 'le'], [4644, 'les'], [3129, 'qui'], [2929, 'e'], [2519, 'des'], [2414, 'ne'], [2360, 'pour'], [2321, 'en'], [2271, 'a'], [2155, 'à'], [1895, 'du'], [1865, 'ce'], [1762, 'l'], [1610, 'par'], [1589, 'vous'], [1456, 'il']]


In [55]:
l_diff = [[dic_effectifs_global[x], x] for x in manquants]
print(sorted(l_diff, reverse=True)[:20])

[[2502, '1'], [1636, '¬'], [1439, 'qu’il'], [1367, 'ſe'], [1290, 'ſi'], [1229, '.'], [1124, 'ſon'], [1023, 'ſa'], [973, 'eſt'], [874, 'ſes'], [834, 'E'], [820, 'à'], [750, 'A'], [714, ','], [585, 'Et'], [562, 'ſur'], [547, 'ſont'], [490, 'Le'], [485, '2'], [479, '3']]


In [56]:
## un lexique par type de tokenisation, on compare le taux de lexicalié

In [57]:
## classer pages selon longueur et taux lex
##ajouter pages témoins